In [1]:
# !pip install tfds-nightly

In [2]:
# Imports
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_text
from typing import Any

# Only let TensorFlow allocate RAM as needed
physical_devices = tf.config.experimental.list_physical_devices("GPU")
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Configuration
tf.random.set_seed(0)
batch_size = 64
snli_ds_name = "snli"
sentence_features = ["premise", "hypothesis"]
saved_model_dir = "saved_model"

2022-10-31 05:06:58.986412: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-31 05:06:59.141458: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Load saved model
preprocessor_save_path = os.path.join(saved_model_dir, "preprocessor")
preprocessor = tf.keras.models.load_model(preprocessor_save_path, compile=False)

classifier_save_path = os.path.join(saved_model_dir, "classifier")
classifier = tf.keras.models.load_model(classifier_save_path, compile=False)

2022-10-31 05:07:01.531446: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-31 05:07:02.327632: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22260 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:68:00.0, compute capability: 8.6


In [4]:
def get_data_from_dataset(full_dataset: "dict[Any, tf.data.Dataset]", info: tfds.core.DatasetBuilder, split: str,
                          batch_size: int, preprocessor: tf.keras.Model) -> "tuple[tf.data.Dataset, int]":
    is_training = split.startswith("train")
    dataset = tf.data.Dataset.from_tensor_slices(full_dataset[split])
    num_examples = info.splits[split].num_examples

    if is_training:
        dataset = dataset.shuffle(num_examples)
        dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda ex: (preprocessor(ex), ex["label"]))
    dataset = dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset, num_examples

# Create train, test, and validation data
snli_ds: "dict[tfds.Split, tf.data.Dataset]" = tfds.load(snli_ds_name, batch_size=-1, shuffle_files=True)  # type: ignore
snli_ds_info = tfds.builder(snli_ds_name).info

train_data, train_data_size = get_data_from_dataset(snli_ds, snli_ds_info,
    "train", batch_size, preprocessor)
steps_per_epoch = train_data_size // batch_size

validation_data, validation_data_size = get_data_from_dataset(snli_ds, snli_ds_info,
    "validation", batch_size, preprocessor)
validation_steps = validation_data_size // batch_size

test_data, test_data_size = get_data_from_dataset(snli_ds, snli_ds_info,
    "test", batch_size, preprocessor)

/usr/local/lib/python3.8/dist-packages/keras/engine/functional.py:637: UserWarning: Input dict contained keys ['label'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


In [5]:
optimizer = tf.keras.optimizers.Adam(2e-5)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy("accuracy", dtype=tf.float32)]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile model
classifier.compile(optimizer=optimizer, metrics=metrics, loss=loss)

# Evaluate model
classifier.evaluate(test_data)

  1/157 [..............................] - ETA: 5:00 - loss: 0.6702 - accuracy: 0.7500

2022-10-31 05:07:22.570482: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


157/157 [==============================] - 19s 111ms/step - loss: nan - accuracy: 0.7792


[nan, 0.77920001745224]